In [60]:
import pandas as pd

# Replace 'your_file.csv' with the actual name of your CSV file
df_base_NN = pd.read_csv('Insurance_base_model_NN_2.csv')
df_base_Catboost = pd.read_csv('Insurance_base_model_CatBoost_1.csv')
df_base_LGBM = pd.read_csv('Insurance_base_LGBM_1.csv')

df_test_NN = pd.read_csv('Insurance_test_set_CatBoost_1.csv')
df_test_Catboost = pd.read_csv('Insurance_test_set_NN_2.csv')
df_test_LGBM = pd.read_csv('Insurance_test_set_LGBM_1.csv')

In [61]:
df_base_NN

,y_val_inv,y_pred
0,2742.0,779.01465
1,1347.0,924.40910
2,2196.0,749.88000
3,684.0,787.47687
4,1714.0,897.66125
...,...,...
359995,635.0,778.62463
359996,831.0,776.93420
359997,111.0,229.25645
359998,32.0,766.12670


In [62]:
df_base_Catboost

,y_val_orig,y_pred_val
0,2742.0,949.222833
1,1347.0,987.599187
2,2196.0,716.527053
3,684.0,806.781974
4,1714.0,847.388166
...,...,...
359995,635.0,434.228299
359996,831.0,786.356451
359997,111.0,240.722936
359998,32.0,747.538399


In [63]:
df_base_LGBM

,y_val_cv,y_pred
0,2742.0,846.600499
1,1347.0,955.473481
2,2196.0,784.409131
3,684.0,737.170860
4,1714.0,1035.686314
...,...,...
359995,635.0,845.001424
359996,831.0,766.838983
359997,111.0,216.831539
359998,32.0,845.738301


# Blend: With meta learner model

In [79]:
# X = df_base_NN
# X['y_pred_catboost'] = df_base_Catboost['y_pred_val']
# X

In [80]:
# # Rename columns in X

# X = X.rename(columns={'y_val_inv': 'y_val', 'y_pred': 'y_pred_NN', 'y_pred_catboost': 'y_pred_catboost'})
# X

In [81]:
# # Create a ridge regression model and train it on X

# from sklearn.linear_model import Ridge

# # Assuming X is your DataFrame and 'y_val' is your target variable
# y = X['y_val']
# X = X.drop('y_val', axis=1)

# # Create and train the Ridge regression model
# ridge_model = Ridge(alpha=1.0)  # You can adjust the alpha parameter
# ridge_model.fit(X, y)

In [26]:
df_test_NN

,id,Premium Amount
0,1200000,940.947511
1,1200001,897.489011
2,1200002,840.769329
3,1200003,771.457641
4,1200004,838.687708
...,...,...
799995,1999995,1003.813121
799996,1999996,696.718840
799997,1999997,891.002356
799998,1999998,801.464635


In [27]:
df_test_Catboost

,id,Premium Amount
0,1200000,882.63940
1,1200001,795.94160
2,1200002,814.86426
3,1200003,788.60260
4,1200004,784.86670
...,...,...
799995,1999995,912.93854
799996,1999996,704.80270
799997,1999997,794.66120
799998,1999998,817.99805


In [82]:
df_test_LGBM

,id,Premium Amount
0,1200000,669.256146
1,1200001,761.214766
2,1200002,801.085050
3,1200003,820.130558
4,1200004,734.131095
...,...,...
799995,1999995,979.003823
799996,1999996,601.473152
799997,1999997,833.106948
799998,1999998,758.934617


In [83]:
# df_test_NN['y_pred_catboost'] = df_test_Catboost['Premium Amount']
# df_test_NN

In [84]:
# df_test_NN = df_test_NN.rename(columns={'Premium Amount': 'y_pred_NN'})
# df_test_NN

In [85]:
# # prompt: ridge_model to predict

# # Use the trained ridge model to make predictions on the test set
# y_pred_ridge = ridge_model.predict(df_test_NN.drop('id', axis=1))
# y_pred_ridge

In [86]:
# # Generate IDs from 1200000 to 1999999 (matching the length of predictions)
# start_id = 1200000
# end_id = start_id + len(y_pred_ridge)

# # Create a DataFrame for submission
# submission_df = pd.DataFrame({
#     'id': range(start_id, end_id),
#     'Premium Amount': y_pred_ridge
# })

# # Save the predictions to a CSV file
# submission_df.to_csv('Insurance_test_blend_1.csv', index=False)
# print(submission_df.head())

# # Submission score: 1.06823 (Current best NN score)

# Blend: Weight Optimization

In [66]:
import pandas as pd

# 1) Create the "train" DataFrame
train = pd.DataFrame({
    'TARGET': df_base_NN['y_val_inv'],          # The actual target values in your validation or CV set
    'catb': df_base_Catboost['y_pred_val'],     # CatBoost predictions on CV
    # 'nn': df_base_NN['y_pred'],              # Neural Net predictions on CV
    'LGBM': df_base_LGBM['y_pred']           # LGBM predictions on CV
})

# 2) Create the "test" DataFrame
test = pd.DataFrame({
    'catb': df_test_Catboost['Premium Amount'],   # CatBoost predictions on test set
    # 'nn': df_test_NN['Premium Amount']            # Neural Net predictions on test set
    'LGBM': df_test_LGBM['Premium Amount']         # LGBM predictions on test set
})

In [67]:
train

,TARGET,catb,LGBM
0,2742.0,949.222833,846.600499
1,1347.0,987.599187,955.473481
2,2196.0,716.527053,784.409131
3,684.0,806.781974,737.170860
4,1714.0,847.388166,1035.686314
...,...,...,...
359995,635.0,434.228299,845.001424
359996,831.0,786.356451,766.838983
359997,111.0,240.722936,216.831539
359998,32.0,747.538399,845.738301


In [68]:
test

,catb,LGBM
0,882.63940,669.256146
1,795.94160,761.214766
2,814.86426,801.085050
3,788.60260,820.130558
4,784.86670,734.131095
...,...,...
799995,912.93854,979.003823
799996,704.80270,601.473152
799997,794.66120,833.106948
799998,817.99805,758.934617


In [69]:
# members = ['catb', 'nn']
members = ['catb', 'LGBM']

In [70]:
from sklearn.metrics import root_mean_squared_log_error

def LOSS(y_true, y_pred):
    return root_mean_squared_log_error(y_true, np.clip(y_pred, 20, 4999))

def SCORE(y_true, y_pred):
    return root_mean_squared_log_error(y_true, y_pred)

In [71]:
from functools import partial
import scipy as sp

class Averager(object):

    def __init__(self, method='nelder-mead', round_avg=False, options={}):
        self.weights_ = []
        self.opt_ = ''
        self.method_ = method
        self.round_avg_ = round_avg
        self.options_ = options

    def _weighted_average(self, weights, values):
        qty = len(values)
        sum_values = values[0] * weights[0]
        sum_weights = weights[0]
        for i in range(1, qty):
            sum_values += values[i] * weights[i]
            sum_weights += weights[i]
        if self.round_avg_:
            return int(np.round(sum_values / sum_weights, 0))
        return sum_values / sum_weights

    def _score(self, weights, values, true_labels):
        preds = self._weighted_average(weights, values)
        return LOSS(true_labels, preds)

    def fit(self, values, true_labels):
        qty = len(values)
        initial_weights = [1 for _ in range(qty)]
        score_partial = partial(self._score, values=values, true_labels=true_labels)
        self.opt_ = sp.optimize.minimize(score_partial, initial_weights, method=self.method_, options=self.options_)
        self.weights_ = self.opt_['x']

    def predict(self, values):
        assert len(self.weights_) == len(values), 'Averager error, must be fitted before predict'
        return self._weighted_average(self.weights_, values)

    def fit_predict(self, values, true_labels):
        self.fit(values, true_labels)
        return self.predict(values)

    def weights(self):
        return self.weights_

    def optimization(self):
        return self.opt_

In [72]:
import numpy as np

def blend(train, test, members, verbose=1):
    # 1) Extract each model’s predictions from train
    X = [train[col].values for col in members]
    if verbose > 0:
        print('train arrays:')
        for a in X:
            print(a)

    # 2) Some iteration limit based on members
    maxiter = len(members) * 500

    # 3) Extract the true target
    y = train['TARGET'].values

    # 4) Fit an Averager (we assume you have the Averager class or library available)
    averager = Averager(options={'maxiter': maxiter})

    # 5) Blend the CV predictions and clip
    x_valid = averager.fit_predict(X, y)
    x_valid = np.clip(x_valid, 20.0, 4999.0)

    if verbose > 0:
        print('\nWEIGHTS:\n', averager.weights())
        print('weighted average:\n', x_valid)

    # 6) Evaluate score on your CV
    score = SCORE(y, x_valid)
    if verbose > 0:
        print(f'estimated SCORE : {score}\n')

    # 7) Blend the test set predictions
    X_test = [test[col].values for col in members]
    if verbose > 0:
        print('arrays to blend:')
        for a in X_test:
            print(a)

    result = averager.predict(X_test)
    result = np.clip(result, 20, 4999)
    if verbose > 0:
        print('weighted average:\n', result, '\n')

    return result, score


In [74]:
blended_test_preds, cv_score = blend(train, test, members=['catb','LGBM'], verbose=1)

train arrays:
[949.22283262 987.59918692 716.52705334 ... 240.72293568 747.53839903
 699.68785605]
[846.60049857 955.47348115 784.40913135 ... 216.83153859 845.73830131
 736.66910011]

WEIGHTS:
 [1.82354594 0.0903099 ]
weighted average:
 [944.3803505  986.08325808 719.73023277 ... 239.59556254 752.17219805
 701.43290517]
estimated SCORE : 1.0329190343238908

arrays to blend:
[882.6394  795.9416  814.86426 ... 794.6612  817.99805 757.7931 ]
[669.25614594 761.21476633 801.08505015 ... 833.10694754 758.93461684
 779.0416917 ]
weighted average:
 [872.57039697 794.30293067 814.21405478 ... 796.47535524 815.21099958
 758.795766  ] 



In [75]:
blended_test_preds

array([872.57039697, 794.30293067, 814.21405478, ..., 796.47535524,
       815.21099958, 758.795766  ])

In [76]:
len(blended_test_preds)

800000

In [54]:
# CatBoost + NN

# Generate IDs from 1200000 to 1999999 (matching the length of predictions)
start_id = 1200000
end_id = start_id + len(blended_test_preds)

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': range(start_id, end_id),
    'Premium Amount': blended_test_preds
})

# Save the predictions to a CSV file
submission_df.to_csv('Insurance_test_blend_2.csv', index=False)
print(submission_df.head())

        id  Premium Amount
0  1200000      881.606738
1  1200001      794.143151
2  1200002      814.405470
3  1200003      788.906245
4  1200004      783.913507


In [77]:
# CatBoost + LGBM

# Generate IDs from 1200000 to 1999999 (matching the length of predictions)
start_id = 1200000
end_id = start_id + len(blended_test_preds)

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': range(start_id, end_id),
    'Premium Amount': blended_test_preds
})

# Save the predictions to a CSV file
submission_df.to_csv('Insurance_test_blend_3.csv', index=False)
print(submission_df.head())

        id  Premium Amount
0  1200000      872.570397
1  1200001      794.302931
2  1200002      814.214055
3  1200003      790.090323
4  1200004      782.472618
